## Estimating Default Probabilities

In [1]:
import numpy as np
import pandas as pd
import locale
from math import exp

In [2]:
locale.setlocale(locale.LC_ALL, '')

'en_US.UTF-8'

In [3]:
class  CashflowDescriptor:
    '''
    Represents cashflow schedules
    '''
    def  __init__(self, coupon_rate, coupon_frequency, notional, T):
        '''
        :param coupon_rate: coupon rate per annum
        :param coupon_frequency: how many times a year is coupon paid
        :param notional: notional amount due
        :param T: time when the last coupon and notional are due
        '''
        self.coupon_rate = coupon_rate
        self.coupon_frequency = coupon_frequency
        self.notional = notional
        self.T = T
        self.coupon_interval = 1 / coupon_frequency
        self.timeline = np.arange(self.coupon_interval, T+self.coupon_interval, self.coupon_interval)
        self.coupon_amount = notional * coupon_rate * self.coupon_interval
        
    def cashflow(self, t):
        if t in self.timeline:  return  self.coupon_amount + (self.notional if t == self.T else 0)
        else:                   return  0
        
    def pv_cashflows_from_time(self, start_time, discount_rate):
        start = start_time if start_time in self.timeline else self.timeline[self.timeline.searchsorted(start_time)]
        timeline = np.arange(start, self.T+self.coupon_interval, self.coupon_interval)
        return  self.pv_cashflows(timeline, discount_rate, t0=start_time)
        
    def pv_cashflows(self, timeline, discount_rate, t0=0):
        return  sum(map(lambda t: self.cashflow(t) * exp(-discount_rate*(t-t0)), timeline))
    
    def pv_all_cashflows(self, discount_rate, t0=0):
        return  self.pv_cashflows(self.timeline, discount_rate, t0)

#### Exercise 19.14
Answer from the textbook:
* The implied pobability of default is 2.74% per year

In [4]:
coupon_rate = .04
coupon_frequency = 2  # semiannual payments
risk_free_rate = .03  # continuous compounding
ytm_rate = .05        # continuous compounding
recovery_rate = .3
T = 4
notional = 100

cashflow_descr = CashflowDescriptor(coupon_rate, coupon_frequency, notional, T)

In [5]:
pv_risk_free_bond = cashflow_descr.pv_all_cashflows(risk_free_rate)
pv_corp_bond      = cashflow_descr.pv_all_cashflows(ytm_rate)
print("Present value risk free bond: %s" % locale.currency(pv_risk_free_bond, grouping=True))
print("Present value corporate bond: %s" % locale.currency(pv_corp_bond, grouping=True))

Present value risk free bond: $103.66
Present value corporate bond: $96.19


In [6]:
tl = pd.Series(range(1, cashflow_descr.T+1)) # Defaults can only take place immediately before coupon payment at year's end
table = pd.DataFrame(index=tl, columns=['Recovery Amount', 'Default-Free Value', 'Loss', 'PV of Expected Loss'])
table.index.name = 'Time'

In [7]:
table['Recovery Amount'] = cashflow_descr.notional * recovery_rate
table['Default-Free Value'] = table.index.to_series().apply(lambda x: cashflow_descr.pv_cashflows_from_time(x, risk_free_rate))
table['Loss'] = table['Default-Free Value'] - cashflow_descr.notional * recovery_rate
table['PV of Expected Loss'] = table.apply(lambda row: exp(-risk_free_rate*row.name)*row.Loss, axis=1)

In [8]:
table

,Recovery Amount,Default-Free Value,Loss,PV of Expected Loss
Time,,,,
1,30.0,104.783107,74.783107,72.572932
2,30.0,103.883092,73.883092,69.580476
3,30.0,102.955668,72.955668,66.676460
4,30.0,102.000000,72.000000,63.858271


In [9]:
Q = (pv_risk_free_bond - pv_corp_bond) / table['PV of Expected Loss'].sum()
print("Risk-neutral annual default probability: %.5f%%" % (100*Q))

Risk-neutral annual default probability: 2.73661%


#### Exercise 19.15
Answer from the textbook:
* 
* 

In [10]:
coupon_rate = .04
coupon_frequency = 1  # semiannual payments
risk_free_rate = .035 # continuous compounding
ytm1_rate = .045      # continuous compounding
ytm2_rate = .0475     # continuous compounding
recovery_rate = .4
T1 = 3
T2 = 5
notional = 100
default_year_offset = .5 # defaults take place halfway through each year

cashflow_descr1 = CashflowDescriptor(coupon_rate, coupon_frequency, notional, T1)
cashflow_descr2 = CashflowDescriptor(coupon_rate, coupon_frequency, notional, T2)

In [11]:
pv_risk_free_bond1 = cashflow_descr1.pv_all_cashflows(risk_free_rate)
pv_corp_bond1      = cashflow_descr1.pv_all_cashflows(ytm1_rate)
print("Present value risk free %d-year bond: %s" % (cashflow_descr1.T, locale.currency(pv_risk_free_bond1, grouping=True)))
print("Present value %d-year corporate bond: %s" % (cashflow_descr1.T, locale.currency(pv_corp_bond1, grouping=True)))

pv_risk_free_bond2 = cashflow_descr2.pv_all_cashflows(risk_free_rate)
pv_corp_bond2      = cashflow_descr2.pv_all_cashflows(ytm2_rate)
print("Present value risk free %d-year bond: %s" % (cashflow_descr2.T, locale.currency(pv_risk_free_bond2, grouping=True)))
print("Present value %d-year corporate bond: %s" % (cashflow_descr2.T, locale.currency(pv_corp_bond2, grouping=True)))

Present value risk free 3-year bond: $101.23
Present value 3-year corporate bond: $98.35
Present value risk free 5-year bond: $101.97
Present value 5-year corporate bond: $96.24


In [12]:
tl1 = pd.Series(np.arange(default_year_offset, cashflow_descr1.T, cashflow_descr1.coupon_interval)) # Defaults can only take place immediately before coupon payment at year's end
table1 = pd.DataFrame(index=tl1, columns=['Recovery Amount', 'Default-Free Value', 'Loss', 'PV of Expected Loss'])
table1.index.name = 'Time'

In [13]:
table1['Recovery Amount'] = cashflow_descr1.notional * recovery_rate
table1['Default-Free Value'] = table1.index.to_series().apply(lambda x: cashflow_descr1.pv_cashflows_from_time(x, risk_free_rate))
table1['Loss'] = table1['Default-Free Value'] - cashflow_descr1.notional * recovery_rate
table1['PV of Expected Loss'] = table1.apply(lambda row: exp(-risk_free_rate*row.name)*row.Loss, axis=1)

In [14]:
table1

,Recovery Amount,Default-Free Value,Loss,PV of Expected Loss
Time,,,,
0.5,40.0,103.012789,63.012789,61.919658
1.5,40.0,102.611458,62.611458,59.409153
2.5,40.0,102.195833,62.195833,56.984995


In [15]:
Q1 = (pv_risk_free_bond1 - pv_corp_bond1) / table1['PV of Expected Loss'].sum()

In [16]:
tl2 = pd.Series(np.arange(default_year_offset, cashflow_descr2.T, cashflow_descr2.coupon_interval)) # Defaults can only take place immediately before coupon payment at year's end
table2 = pd.DataFrame(index=tl2, columns=['Recovery Amount', 'Default-Free Value', 'Loss', 'PV of Expected Loss'])
table2.index.name = 'Time'

In [17]:
table2['Recovery Amount'] = cashflow_descr2.notional * recovery_rate
table2['Default-Free Value'] = table2.index.to_series().apply(lambda x: cashflow_descr2.pv_cashflows_from_time(x, risk_free_rate))
table2['Loss'] = table2['Default-Free Value'] - cashflow_descr2.notional * recovery_rate
table2['PV of Expected Loss'] = table2.apply(lambda row: exp(-risk_free_rate*row.name)*row.Loss, axis=1)

In [18]:
table2

,Recovery Amount,Default-Free Value,Loss,PV of Expected Loss
Time,,,,
0.5,40.0,103.774514,63.774514,62.668169
1.5,40.0,103.400316,63.400316,60.157664
2.5,40.0,103.012789,63.012789,57.733506
3.5,40.0,102.611458,62.611458,55.392727
4.5,40.0,102.195833,62.195833,53.132458


In [19]:
PV1 = table2['PV of Expected Loss'].iloc[:cashflow_descr1.T].sum()
PV2 = table2['PV of Expected Loss'].iloc[cashflow_descr1.T:].sum()
print(PV1, PV2)

180.55933866522133 108.52518451951184


In [20]:
Q2 = (pv_risk_free_bond2 - pv_corp_bond2 - Q1*PV1) / PV2

In [21]:
print("Q1 (Risk-neutral annual default probability for first %d years): %.5f%%" % (cashflow_descr1.T, 100*Q1))
print("Q2 (Risk-neutral annual default probability for last %d years): %.5f%%"
      % (cashflow_descr2.T-cashflow_descr1.T, 100*Q2))

Q1 (Risk-neutral annual default probability for first 3 years): 1.61489%
Q2 (Risk-neutral annual default probability for last 2 years): 2.59462%
